In [19]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import pandas as pd
import faiss                   # make faiss available
import umap
import seaborn as sns
%matplotlib inline
from byol_pytorch import BYOL

import os
import sys
import matplotlib.pyplot as plt
from sklearn import svm
sys.path.append('..')

import torchvision.models as models
from torchvision import transforms
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE

from pl_bolts.models.self_supervised import SimCLR
import lightgbm as lgb
import kornia
from kornia.augmentation.container import AugmentationSequential

import torchvision.transforms.functional as tvF
from src.beam import UniversalDataset, Experiment, Algorithm, beam_arguments, PackedFolds, batch_augmentation
from src.beam import tqdm, as_numpy
from examples.ssl_with_beam import STL10Dataset, my_ssl_algorithm, ImageNetAugmented

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
experiment = Experiment.reload_from_path("/home/shared/data/results/beam_ssl/BeamVICReg/resnet_parallel/0000_20220823_132503", 
                                         override_hparams={'device': 2, 'lgb_device': None,
                                                           'lgb_rounds': 40, 'lgb_num_leaves': 31,
                                                           'lgb_max_depth': 4,
                                                           'verbose_lgb': False,})


2022-09-21 10:38:02 | INFO | Reload experiment from path: /home/shared/data/results/beam_ssl/BeamVICReg/resnet_parallel/0000_20220823_132503
2022-09-21 10:38:02 | INFO | Resuming existing experiment
2022-09-21 10:38:02 | INFO | Reload experiment from checkpoint: /home/shared/data/results/beam_ssl/BeamVICReg/resnet_parallel/0000_20220823_132503/checkpoints/checkpoint_001811


In [21]:
dataset_unlabeled = STL10Dataset(experiment.hparams, subset='unlabeled')
dataset_labeled = STL10Dataset(experiment.hparams, subset='labeled')

/opt/conda/lib/python3.8/site-packages/kornia/augmentation/_2d/intensity/color_jitter.py:83: DeprecationWarning: `ColorJitter` is now following Torchvision implementation. Old behavior can be retrieved by instantiating `ColorJiggle`.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/kornia/augmentation/_2d/intensity/color_jitter.py:83: DeprecationWarning: `ColorJitter` is now following Torchvision implementation. Old behavior can be retrieved by instantiating `ColorJiggle`.
  warnings.warn(


In [22]:
from src.beam.server import BeamClient

In [23]:
client = BeamClient('localhost:46664')

In [24]:
info = client.get('')

In [29]:
preds = client.post('alg/predict', dataset_labeled[400:500])

In [28]:
preds

DataBatch(index=tensor([400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413,
        414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427,
        428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441,
        442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455,
        456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469,
        470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483,
        484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497,
        498, 499]), data={'y': tensor([ 3,  8,  5, 10,  8,  7,  4,  6,  1,  6,  5,  8,  1,  8, 10,  7,  2,  9,
         3,  3,  3,  2,  1, 10,  6,  2, 10,  3,  8,  3,  1, 10,  2,  7,  5,  6,
         6,  1,  7,  3,  8,  3,  4,  3,  1,  6,  9,  8,  9,  9,  6,  6,  6,  9,
         8,  4, 10,  2, 10,  2,  1,  7,  4,  7,  2,  7,  1,  6,  3,  8,  8,  2,
         9,  7,  8,  7,  4,  8,  1, 10,  2,  9,  6,  9,  9, 10,  5,  1, 10,  2,
       